In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Vertex AI：使用Vertex ML Metadata在Vertex AI Pipelines运行中跟踪工件和指标

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/ml_metadata/vertex-pipelines-ml-metadata.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/ml_metadata/vertex-pipelines-ml-metadata.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/ml_metadata/vertex-pipelines-ml-metadata.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
     </a>
     </td>
</table>

## 概述

本笔记本演示了如何跟踪 Vertex AI 管道运行中的指标和工件，并使用 Vertex AI SDK for Python 分析这些元数据。如果您更喜欢按步骤进行教程，请查看本笔记本的[codelab 版本](https://codelabs.developers.google.com/vertex-mlmd-pipelines#0)。

了解更多关于[Vertex ML Metadata](https://cloud.google.com/vertex-ai/docs/ml-metadata)和[Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction)。

### 目标

在这个笔记本中，您将学习如何使用Vertex ML Metadata在Vertex AI Pipeline运行中跟踪工件和指标。

本教程使用以下谷歌云ML服务和资源：

- Vertex AI Pipelines
- Vertex ML Metadata

执行的步骤包括：

* 使用Kubeflow Pipelines SDK构建在Vertex AI上运行的ML管道。
* 该管道创建数据集，训练一个scikit-learn模型，并将模型部署到一个端点。
* 编写自定义管道组件来生成工件和元数据。
* 比较Vertex AI Pipelines运行，无论是在谷歌云控制台还是以编程方式。
* 跟踪管道生成的工件的谱系。
* 查询您的管道运行元数据。

数据集

在这份笔记本中，您将使用scikit-learn训练模型，通过UCI机器学习中的[干豆数据集](https://archive.ics.uci.edu/ml/datasets/Dry+Bean+Dataset)对豆类进行分类。这是一个包含七种不同类型豆子的测量和特征的表格数据集，这些数据是从图像中获取的。

成本

本教程使用Google Cloud的计费组件：

* Vertex AI
* 云存储

了解[Vertex AI价格](https://cloud.google.com/vertex-ai/pricing)和[云存储价格](https://cloud.google.com/storage/pricing)，并使用[Pricing计算器](https://cloud.google.com/products/calculator/)根据您的预期使用情况生成成本估算。

### 安装额外的软件包

运行以下命令安装Python的Vertex AI SDK和这个笔记本中使用的软件包。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 'kfp<2.0'

### 仅限合作：取消注释以下单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

## 开始之前

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下步骤：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 验证您的Google Cloud帐户

根据您的Jupyter环境，您可能需要手动进行验证。请按照以下相关说明操作。

**1. Vertex AI Workbench**
* 不用做任何操作，因为您已经验证过了。

**2. 本地JupyterLab实例，取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 进行协作，取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

4. 服务账号或其他
* 请查看如何向您的服务账号授予云存储权限的方法，网址为 https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples。

启用此笔记本中使用的云服务。

运行下面的单元格以启用计算引擎、容器注册表和Vertex AI服务。

In [ ]:
!gcloud services enable compute.googleapis.com         \
                       containerregistry.googleapis.com  \
                       aiplatform.googleapis.com

创建一个云存储桶

创建一个存储桶，用于存储诸如数据集等中间产物。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有当您的存储桶尚未存在时：运行以下单元格以创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

### 服务账号

使用服务账号来创建 Vertex AI Pipeline 作业。

如果您不想使用您项目的计算引擎服务账号，将`SERVICE_ACCOUNT`设置为另一个服务账号ID。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
import sys

IS_COLAB = "google.colab" in sys.modules
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

设置Vertex AI管道的服务帐户访问权限

运行以下命令，为您的服务帐户授予读取和写入管道工件的访问权限，该工件位于您在上一步中创建的存储桶中--您只需要为每个服务帐户运行一次这些命令。

In [ ]:
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectCreator $BUCKET_URI

! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.objectViewer $BUCKET_URI

导入库并定义常量

导入所需的库。

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
# We'll use this beta library for metadata querying
from google.cloud import aiplatform, aiplatform_v1beta1
from google.cloud.aiplatform import pipeline_jobs
from kfp.v2 import compiler, dsl
from kfp.v2.dsl import (Artifact, Dataset, Input, Metrics, Model, Output,
                        OutputPath, component)

定义一些常数

In [ ]:
PATH = get_ipython().run_line_magic("env", "PATH")
%env PATH={PATH}:/home/jupyter/.local/bin
REGION = "us-central1"

PIPELINE_ROOT = f"{BUCKET_URI}/pipeline_root/"
PIPELINE_ROOT

初始化Vertex AI SDK

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

## 概念

为了更好地了解 [Vertex AI Pipelines](https://cloud.google.com/vertex-ai/docs/pipelines/introduction) 和 [Vertex AI ML Metadata](https://cloud.google.com/vertex-ai/docs/ml-metadata)，以下是一些相关的概念：

### 管道运行
术语“运行”指的是在Vertex AI管道中执行一次管道的过程。每次运行会生成工件、指标和相关的元数据。

### 工件

工件是由您的管道生成的资源。 工件可以是数据集、模型、端点或在您的管道中定义的自定义资源。

指标

指标是用来衡量管道运行和工件性能的方式。例如，一个指标可以是在您的管道中创建的分类模型工件的准确度，或用于训练模型的数据集大小。

元数据

元数据描述了由您的管道运行生成的工件和度量。例如，模型的元数据可能包括模型工件的URL、名称以及创建时间。

创建具有自定义组件的3步管道

这个实验室的重点是了解管道运行中的元数据。为了做到这一点，您将需要在Vertex AI管道上运行一个管道，这将是您的起点。在这里，您将定义一个包含以下自定义组件的3步管道：

* `get_dataframe`：从一个BigQuery表检索数据并将其转换为一个pandas DataFrame。
* `train_sklearn_model`：使用pandas DataFrame来训练并导出一个scikit-learn模型，还有一些指标。
* `deploy_model`：将导出的scikit-learn模型部署到Vertex AI中的一个端点。

### 创建并定义基于Python函数的组件

首先，用以下代码定义`get_dataframe`组件。该组件执行以下操作：
* 使用BigQuery客户端库创建对BigQuery表的引用
* 下载BigQuery表并将其转换为打乱顺序的pandas DataFrame
* 将DataFrame导出到CSV文件

In [ ]:
@component(
    packages_to_install=["google-cloud-bigquery", "pandas", "pyarrow"],
    base_image="python:3.9",
    output_component_file="create_dataset.yaml",
)
def get_dataframe(bq_table: str, output_data_path: OutputPath("Dataset")):
    from google.cloud import bigquery

    bqclient = bigquery.Client(project=PROJECT_ID)
    table = bigquery.TableReference.from_string(bq_table)
    rows = bqclient.list_rows(table)
    dataframe = rows.to_dataframe(
        create_bqstorage_client=True,
    )
    dataframe = dataframe.sample(frac=1, random_state=2)
    dataframe.to_csv(output_data_path)

接下来，创建一个组件来训练 scikit-learn 模型。这个组件会执行以下操作：
* 导入 CSV 文件作为 pandas DataFrame。
* 将 DataFrame 分割成训练集和测试集。
* 训练一个 scikit-learn 模型。
* 记录模型的指标。
* 将模型保存为本地的 `model.joblib` 文件。

In [ ]:
@component(
    packages_to_install=["scikit-learn", "pandas", "joblib"],
    base_image="python:3.9",
    output_component_file="beans_model_component.yaml",
)
def sklearn_train(
    dataset: Input[Dataset], metrics: Output[Metrics], model: Output[Model]
):
    import pandas as pd
    from joblib import dump
    from sklearn.model_selection import train_test_split
    from sklearn.tree import DecisionTreeClassifier

    df = pd.read_csv(dataset.path)
    labels = df.pop("Class").tolist()
    data = df.values.tolist()
    x_train, x_test, y_train, y_test = train_test_split(data, labels)

    skmodel = DecisionTreeClassifier()
    skmodel.fit(x_train, y_train)
    score = skmodel.score(x_test, y_test)
    print("accuracy is:", score)

    metrics.log_metric("accuracy", (score * 100.0))
    metrics.log_metric("framework", "Scikit Learn")
    metrics.log_metric("dataset_size", len(df))
    dump(skmodel, model.path + ".joblib")

最后，最后一个组件将在前一步中训练得到的模型上传到Vertex AI，并部署到端点：

In [ ]:
@component(
    packages_to_install=["google-cloud-aiplatform"],
    base_image="python:3.9",
    output_component_file="beans_deploy_component.yaml",
)
def deploy_model(
    model: Input[Model],
    project: str,
    region: str,
    vertex_endpoint: Output[Artifact],
    vertex_model: Output[Model],
):
    from google.cloud import aiplatform

    aiplatform.init(project=project, location=region)

    deployed_model = aiplatform.Model.upload(
        display_name="beans-model-pipeline",
        artifact_uri=model.uri.replace("model", ""),
        serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.0-24:latest",
    )
    endpoint = deployed_model.deploy(machine_type="n1-standard-4")

    # Save data to the output params
    vertex_endpoint.uri = endpoint.resource_name
    vertex_model.uri = deployed_model.resource_name

### 定义并编译管道

In [ ]:
@dsl.pipeline(
    # Default pipeline root. You can override it when submitting the pipeline.
    pipeline_root=PIPELINE_ROOT,
    # A name for the pipeline.
    name="mlmd-pipeline",
)
def pipeline(
    bq_table: str = "",
    output_data_path: str = "data.csv",
    project: str = PROJECT_ID,
    region: str = REGION,
):
    dataset_task = get_dataframe(bq_table)

    model_task = sklearn_train(dataset_task.output)

    deploy_model(model=model_task.outputs["model"], project=project, region=region)

以下内容生成一个 JSON 文件，您将使用它来运行管道：

In [ ]:
compiler.Compiler().compile(pipeline_func=pipeline, package_path="mlmd_pipeline.json")

开始两个流水线运行

接下来，您将启动两次我们的流水线运行。首先，定义一个时间戳用于我们流水线作业的ID：

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

当您运行管道时，管道需要一个参数：我们希望用于训练数据的 `bq_table`。这个管道运行使用豆类数据集的一个较小版本。

In [ ]:
run1 = pipeline_jobs.PipelineJob(
    display_name="mlmd-pipeline",
    template_path="mlmd_pipeline.json",
    job_id="mlmd-pipeline-small-{}".format(TIMESTAMP),
    parameter_values={"bq_table": "sara-vertex-demos.beans_demo.small_dataset"},
    enable_caching=True,
)

接下来，使用相同数据集的较大版本创建另一个管道运行。

In [ ]:
run2 = pipeline_jobs.PipelineJob(
    display_name="mlmd-pipeline",
    template_path="mlmd_pipeline.json",
    job_id="mlmd-pipeline-large-{}".format(TIMESTAMP),
    parameter_values={"bq_table": "sara-vertex-demos.beans_demo.large_dataset"},
    enable_caching=True,
)

最后，启动两次运行的管道执行。最好在两个单独的笔记本单元格中进行此操作，以便您可以查看每次运行的输出。

In [ ]:
run1.submit()

然后，开始第二轮：

In [ ]:
run2.submit()

运行完这个单元格后，您将看到一个链接，可以在 Google Cloud 控制台中查看每个流水线。打开该链接以查看有关您的流水线的更多详细信息。

**这些流水线运行将需要10-15分钟才能完成。**

比较管道运行

现在您已经完成了两个流水线运行，可以使用Vertex AI SDK for Python仔细查看流水线指标。

**有关在Google Cloud控制台中查看流水线工件和元数据的指导，请参阅[这个codelab](https://codelabs.developers.google.com/vertex-mlmd-pipelines#5)。**

您可以使用`aiplatform.get_pipeline_df()`方法来访问运行元数据。在这里，您将获取相同管道的最后两次运行的元数据，并将其加载到Pandas DataFrame中。这里的`mlmd-pipeline`参数是指您在管道定义中为管道设置的名称：

In [ ]:
df = aiplatform.get_pipeline_df(pipeline="mlmd-pipeline")
print(df)

你只在这里执行了管道两次，但想象一下，如果执行次数更多，将会有多少指标。接下来，使用matplotlib创建一个定制的可视化图表，来查看模型准确性与用于训练的数据量之间的关系。运行以下命令生成图表：

In [ ]:
plt.plot(df["metric.dataset_size"], df["metric.accuracy"], label="Accuracy")
plt.title("Accuracy and dataset size")
plt.legend(loc=4)
plt.show()

查询管道指标

除了获取所有管道指标的DataFrame之外，您可能希望以编程方式查询在您的ML系统中创建的工件。从那里，您可以创建自定义仪表板，或让您组织中的其他人获取特定工件的详细信息。

获取所有模型工件

要以这种方式查询工件，您将创建一个 `MetadataServiceClient`：

In [ ]:
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)
metadata_client = aiplatform_v1beta1.MetadataServiceClient(
    client_options={"api_endpoint": API_ENDPOINT}
)

接下来，向该端点发出一个`list_artifacts`请求，并传递一个筛选器，指示您想要在响应中的哪些工件。首先，让我们获取项目中所有的**模型**工件。为此，请在您的笔记本中运行以下命令：

In [ ]:
MODEL_FILTER = 'schema_title = "system.Model"'
artifact_request = aiplatform_v1beta1.ListArtifactsRequest(
    parent="projects/{}/locations/{}/metadataStores/default".format(PROJECT_ID, REGION),
    filter=MODEL_FILTER,
)
model_artifacts = metadata_client.list_artifacts(artifact_request)

生成的 `model_artifacts` 响应包含您项目中每个模型工件的可迭代对象，以及每个模型的相关元数据。

### 对象过滤并在DataFrame中显示

接下来，获取所有在2021年8月10日之后创建的具有`LIVE`状态的文物。运行这个请求之后，在一个pandas DataFrame中显示结果。首先，执行这个请求：

In [ ]:
LIVE_FILTER = 'create_time > "2021-08-10T00:00:00-00:00" AND state = LIVE'
artifact_req = {
    "parent": "projects/{}/locations/{}/metadataStores/default".format(
        PROJECT_ID, REGION
    ),
    "filter": LIVE_FILTER,
}
live_artifacts = metadata_client.list_artifacts(artifact_req)

然后在一个数据帧中显示结果:

In [ ]:
data = {"uri": [], "createTime": [], "type": []}

for i in live_artifacts:
    data["uri"].append(i.uri)
    data["createTime"].append(i.create_time)
    data["type"].append(i.schema_title)

df = pd.DataFrame.from_dict(data)
print(df)

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

如果您不想删除项目，请按照以下步骤清理您使用的资源：

- 如果您使用了Vertex AI Workbench笔记本来运行此项目，请停止或删除笔记本实例。
- 在Vertex AI中执行的管道运行部署了端点。转到[Google Cloud控制台](https://console.cloud.google.com/vertex-ai/endpoints)删除这些端点。
- 删除您创建的[Cloud Storage存储桶](https://console.cloud.google.com/storage/browser/)。

In [ ]:
import os

try:
    run1.delete()
    run2.delete()
except Exception as e:
    print(e)

delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}